In [ ]:
## 找出今天數量，各種均量
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

def combineStr(r):
    stockId, open, close, high, ma1, ma5, ma20, ma60, ma120, ma240 = r["stockId"], r["open"], r["close"], r["high"], r["ma1"], r["ma5"], r["ma20"], r["ma60"], r["ma120"], r["ma240"]
    kbar = 'R' if (close > open) else 'G'
    return f"{stockId},{kbar},{open},{close},{high},{ma1},{ma5},{ma20},{ma60},{ma120},{ma240}"

##
opens    = data.get("price:開盤價").fillna(0).astype(float).tail(1)      #如果資料有空白自動填0
closes   = data.get("price:收盤價").fillna(0).astype(float).tail(1)
highs    = data.get("price:最高價").fillna(0).astype(float).tail(1)
volumns  = data.get("price:成交股數").fillna(0).astype(float)[250: ]  
v = (volumns/1000).round(0).astype(int)
vMa5 = v.rolling(5, min_periods=1).mean().round(0).astype(int).tail(1)
vMa20 = v.rolling(20, min_periods=1).mean().round(0).astype(int).tail(1)
vMa60 = v.rolling(60, min_periods=1).mean().round(0).astype(int).tail(1)
vMa120 = v.rolling(120, min_periods=1).mean().round(0).astype(int).tail(1)
vMa240 = v.rolling(240, min_periods=1).mean().round(0).astype(int).tail(1)

## 本益比、EPS
pER = data.get('price_earning_ratio:本益比')
#print(pER["2328"])

sss = data.get('trading_attention')
# print(sss["2328"])

vdf = v.iloc[-1:].transpose()
vdf["stock"] = vdf.index
vdf.columns = ["ma1", "stockId"]
vdf["ma5"] = vMa5.iloc[-1:].transpose()
vdf["ma20"] = vMa20.iloc[-1:].transpose()
vdf["ma60"] = vMa60.iloc[-1:].transpose()
vdf["ma120"] = vMa120.iloc[-1:].transpose()
vdf["ma240"] = vMa240.iloc[-1:].transpose()
vdf["open"] = opens.iloc[-1:].transpose()
vdf["close"] = closes.iloc[-1:].transpose()
vdf["high"] = highs.iloc[-1:].transpose()
vdf["comp"] = vdf.apply(combineStr, axis = 1)
vdf2 = vdf[ (vdf["stockId"].str.len() == 4)].dropna(axis = 0, how ='any')
# print(vdf2)

s1 = 'id,kbar,yOpen,yClose,yHigh,yVolume,ma5,ma20,ma60,ma120,ma240\n'
for c in vdf2["comp"].tolist():
    s1 += f"{c}\n"
# print(s1)

from datetime import date, timedelta
import time

today = date.today()                        # 取得今天的日期
yesterday = today - timedelta(days=1)       # 計算昨天的日期
yesterday = yesterday.strftime("%Y%m%d")    # 將日期轉換為指定的格式
print(yesterday)
fm.write_LogFile(f"volumeData/ma_{yesterday}.csv", s1)

In [ ]:
import finlab
finlab.login(open("config.txt", "r").read())

from finlab import data
import pandas as pd
import _beowFmt as fm 

## 本益比、EPS
stockId = "2328"
pER = data.get('price_earning_ratio:本益比')
# print(pER[stockId])

EPS = data.get('fundamental_features:每股稅後淨利')
EPS成長率 = EPS.rolling(4).sum()/EPS.shift(4).rolling(4).sum() #最近四季的EPS/最近八季~四季的EPS總和推算
預估EPS = EPS*EPS成長率

print(EPS[stockId].tail(5))
print(EPS成長率[stockId].tail(5))

print(預估EPS[stockId].tail(5))

In [ ]:
## 自己擷取數量，只用來與昨天比較...
import _beowFmt as fm 
dd = dfb[["id","close","volume"]].sort_values("id")
ss = 'id,close,yVolume\n'
for d in dd.values.tolist():
  ss += f"{d[0]},{d[1]},{d[2]}\n"
fm.write_LogFile(f"volumeData/20230110.csv", ss)

# todo ##
# https://www.wantgoo.com/investrue/all-previous-weekly-candlestick?v=1672966860000  上一周收盤價


In [ ]:
import time
localtime = time.localtime()
nowTime = time.strftime("%H:%M", localtime)
print(nowTime)
# multiple_level_data = pd.json_normalize(data, record_path =['Results'], meta =['original_number_of_clusters','Scaler','family_min_samples_percentage'], meta_prefix='config_params_', record_prefix='dbscan_')

In [ ]:
import _beowFmt as fm 

df2 = df[(df["id"]>="1101") & (df["id"]<="9999") & (df["volume"] >= 1000)].sort_values(by=['id'], ascending=True)
dd = df2[{"id","volume"}]
ss = ''
for d in dd.values.tolist():
  ss += f"{d[1]}, {d[0]}\n"

fm.write_LogFile(f"keepData/stockVolume.csv", ss)

In [ ]:
from pathlib import Path  

filepath = Path('https://www.wantgoo.com/investrue/all-quote-info')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
df.to_csv(filepath)  

In [ ]:
import time

arr = contents.split(",")
arr = [e.replace(".TW", "") for e in arr]
print(arr)

def getKbar(r):
    stockId, open, close = r["id"], r["open"], r["close"]
    return 'R' if (close > open) else 'G'

def getTurnOver(r):
    stockId = r["id"]
    # print((str(stockId) in arr),stockId, arr)
    return 1 if (str(stockId) in arr) else 0 

vdf["kbar"]     = vdf.apply(getKbar, axis = 1)
vdf["turnOver"] = vdf.apply(getTurnOver, axis = 1)
# print(vdf)
vdf2 = vdf[ (vdf["turnOver"] > 0) ].dropna(axis = 0, how ='any')
# print(vdf2)
vdf3 = vdf2[ (vdf["ylow"] > vdf["low"]) & (vdf["close"] > vdf["yclose"]) ].dropna(axis = 0, how ='any')
print(vdf3)

ss , o_nowDate = '', time.strftime("%m%d_1400", time.localtime())
for d in vdf3["id"].tolist():
  ss += f"{d}.TW,"
fm.write_LogFile(f"xq_import/量價型態2023{o_nowDate}_破底拉回.csv", ss)

In [ ]:
import _beowFmt as fm 
lastDate = fm.getLastFileDate("volumeData", "ma_")

#前置作業準備，只要一次!!
import requests
import pandas as pd
from datetime import datetime
from datetime import date
from bs4 import BeautifulSoup
import json

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180) 

## 取股票名稱 ##
dfStockName = pd.read_csv(f"paras/股票名稱.csv")
dfStockName.columns = ["stockId", "中文名稱"]
dfStockName["stockId"] = dfStockName["stockId"].astype('str')

## 設定AB兩點 ##
dft = pd.read_csv(f"stock-homework/tradingLine.csv")
dft = dft.drop("stockName", axis=1)
dft["stockId"] = dft["stockId"].astype('str')
dft["股票價差"] = dft["after"] - dft["before"]
dft["A_Date"] = pd.to_datetime(dft['beforeD'], format='%Y/%m/%d') #A點
dft["B_Date"] = pd.to_datetime(dft['afterD'], format='%Y/%m/%d')   #B點
dft["C_Date"] = pd.to_datetime(dft["today"], format='%Y/%m/%d')    #C點(模擬今天日)
# dft["C_Date"] = pd.to_datetime(pd.Timestamp.now().date())   #C點(真實今天日)

# print(dft)
dft["BA_diff"] = (dft["B_Date"] - dft["A_Date"]).dt.days.astype(int) / dft["timegap"] * dft["fed"]
dft["CA_diff"] = (dft["C_Date"] - dft["A_Date"]).dt.days.astype(int) / dft["timegap"] * dft["fed"]
dft["C_point"] = dft["before"] + (dft["股票價差"] / dft["BA_diff"] * dft["CA_diff"])
# print(dft.dtypes)


import finlab
finlab.login(open("config.txt", "r").read())
from finlab import data
import pandas as pd
import numpy as np
import _beowFmt as fm 

opens    = data.get("price:開盤價").fillna(0).astype(float).tail(1)      #如果資料有空白自動填0
closes   = data.get("price:收盤價").fillna(0).astype(float).tail(1)
highs    = data.get("price:最高價").fillna(0).astype(float).tail(1)
volumns  = data.get("price:成交股數").fillna(0).astype(float)[250: ]  
v        = (volumns/1000).round(0).astype(int)
dfv = v.iloc[-1:].transpose()
dfv["stock"] = dfv.index
dfv.columns = ["ma1", "stockId"]
dfv["open"] = opens.iloc[-1:].transpose()
dfv["close"] = closes.iloc[-1:].transpose()
dfv["high"] = highs.iloc[-1:].transpose()

dfc = pd.merge(dft, dfv, left_on="stockId", right_on="stockId")
dfc = pd.merge(dfc, dfStockName, left_on="stockId", right_on="stockId")
dfc["過高"] = np.where((dfc["close"] > dfc["C_point"]), "V", "")
dfc["K棒壓過"] = np.where((dfc["close"] > dfc["C_point"]) & (dfc["C_point"] > dfc["open"]), "V", "") 
# print(dfc.dtypes)

print(dfc.loc[:, ["timegap","fed","stockId","中文名稱","before","beforeD","after","afterD","C_point","過高", "K棒壓過","open","close","high" ]]) #


In [ ]:
from finlab import data
import pandas as pd
import _beowFmt as fm 
import os

rootpath= "D:/project/stockDataLab"
last1_file_name = fm.getLastFileDate(f"{rootpath}/xq_import", "量價型態_")
last2_file_name = fm.getLast2FileDate(f"{rootpath}/xq_import", "量價型態_")
last_date = last1_file_name.split("_")[0].replace("量價型態", "")
print(last_date)

df1 = pd.read_csv(f"{rootpath}/xq_import/{last1_file_name}.csv")
df2 = pd.read_csv(f"{rootpath}/xq_import/{last2_file_name}.csv")
list1 = df1.columns.tolist()[:-1]
list2 = df2.columns.tolist()[:-1]

diff2 = list(set(list1) - set(list2))

# 將".TW"移出，使用列表解析
stock_diff = [stock.replace('.TW', '') for stock in diff2]

# print(list1)
# print(list2)
# print(diff2)
# print(list1)
# print(stock_diff)

df_diff = pd.DataFrame(stock_diff, columns = ["stockId"])
df_diff["time"] = last1_file_name.removesuffix("_量比大").removeprefix("量價型態2023")

## 取股票名稱 #--------------------------------------------------
dfStockName = pd.read_csv(f"paras/股票名稱.csv")
dfStockName.columns = ["stockId", "中文名稱"]
dfStockName["stockId"] = dfStockName["stockId"].astype('str')
## -------------------------------------------------------------

dfc = pd.merge(df_diff, dfStockName, left_on="stockId", right_on="stockId")
# print(dfc)

csv_file_path = f"{rootpath}/xq_import/周轉率最新_{last_date}.csv"

if not os.path.exists(csv_file_path) or os.path.getsize(csv_file_path) == 0:
  df_combined = pd.DataFrame(columns=['stockId', '中文名稱', 'time']).astype({'stockId': int, '中文名稱': str, 'time': str})
else:
  df_combined = pd.read_csv(csv_file_path, sep='\t')
  # print(df_combined)

df_combined2 = pd.concat([dfc, df_combined], ignore_index=True)
# print(df_combined2)
# 將DataFrame輸出到文字檔
df_combined2.to_csv(csv_file_path, index=False, sep='\t')

In [ ]:
## 整理出唯一的
from finlab import data
import pandas as pd
import _beowFmt as fm 
import os

rootpath= "D:/project/stockDataLab"
last1_file_name = '周轉率最新_20230816'
df1 = pd.read_csv(f"{rootpath}/xq_import/{last1_file_name}.csv", sep='\t')
df2 = df1.drop_duplicates(subset=['stockId', '中文名稱']).sort_values(by=['time','stockId'], ascending=False)
df3 = df2[['time', 'stockId', '中文名稱']]
print(df3)
df3.to_csv(f"{rootpath}/xq_import/{last1_file_name}a.csv", index=False, sep='\t')

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import json
import _beowFmt as fm 
from IPython.display import display

pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option("expand_frame_repr", False)
pd.set_option('display.width', 180)                       # 设置打印宽度(**重要**)
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36(KHTML, like Gecko) Chrome/78.0.3904.97 Safari/537.36"
}

r = requests.get("https://www.tpex.org.tw/web/stock/most_active/active_BROKER_vol/rt_brk_result.php?l=zh-tw", headers = headers).content
soup = BeautifulSoup(r, "html.parser")
rr = soup.prettify()
data = pd.read_json(rr)
data2 = data["aaData"]
# 将每个内部list转换为DataFrame的一行，同时去除首尾空白
df = pd.DataFrame([[item.strip() if isinstance(item, str) else item for item in row] for row in data2],
                  columns=["i", "投信", "買進金額", "賣出金額", "groupId","Stock"])
                  
df["買進金額"] = df["買進金額"].astype(int)
df["賣出金額"] = df["賣出金額"].astype(int)
df[["stockName", "stockId"]] = df["Stock"].str.strip("()").str.split("(", expand=True) # 使用 str.split 将 "Column6" 分割成两列
df["買入金額"] = df["買進金額"] - df["賣出金額"]

df33= df[df["stockId"] == "3483"]
grouped = df.groupby(['stockId', 'stockName'])['買入金額'].sum().reset_index()

sorted_df = grouped.sort_values(by='買入金額', ascending=False)     # 根據買入總金額降序排序

print(sorted_df)
